Goal: Download texts from project gutenberg and visualize differences via plotly (spectral embedding, tsne, umap, etc.)

Requirements:
https://github.com/c-w/gutenberg
Install BerkeleyDB

brew install berkeley-db4

pip install gutenberg

In [1]:
# build a few functions to acquire many books
from gutenberg.query import get_etexts
from gutenberg.query import get_metadata
from gutenberg.cleanup import strip_headers
from gutenberg.acquire import get_metadata_cache
from gutenberg.acquire import load_etext
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from collections import Counter
import numpy as np
import pandas as pd
from umap import UMAP
import nltk
from sklearn.decomposition import KernelPCA as KPCA
from sklearn.pipeline import Pipeline
import plotly.graph_objs as go
import plotly.offline as py

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joshualevy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
cache = get_metadata_cache()
cache.populate()

CacheAlreadyExistsException: location: /Users/joshualevy/gutenberg_data/metadata/metadata.db

In [4]:
possible_idx = []
for i in range(3000):
    try:
        a=list(get_metadata('author', i))[0]
        possible_idx.append(i)
    except:
        pass
possible_idx = np.array(possible_idx)

In [6]:
len(possible_idx)

2880

In [5]:
authors = np.array([(list(get_metadata('author', i))[0],i) for i in possible_idx])
print(dict(Counter(authors[:,0]).most_common(15)).keys()) # about 30 each

dict_keys(['Shakespeare, William', 'Balzac, Honoré de', 'Dickens, Charles', 'Harte, Bret', 'Stevenson, Robert Louis', 'James, Henry', 'London, Jack', 'Galsworthy, John', 'Dumas, Alexandre', 'Goethe, Johann Wolfgang von', 'Huxley, Thomas Henry', 'Casanova, Giacomo', 'Doyle, Arthur Conan', 'Jerome, Jerome K. (Jerome Klapka)', 'Davis, Richard Harding'])


In [12]:
sum(np.isin(authors[:,0],list(dict(Counter(authors[:,0]).most_common(15)).keys())))

709

In [14]:
authors=authors[np.isin(authors[:,0],list(dict(Counter(authors[:,0]).most_common(15)).keys())),:]

In [29]:
load_etext('42')

InvalidEtextIdException: 

In [32]:
txts = {'author':[],'title':[],'text':[]}
for i in log_progress(authors[:,1], every=1):
    metadata = get_metadata('author', i)
    #print(metadata)
    if metadata:
        try:
            author = list(metadata)[0]
            #if i > 200 and author not in dict(Counter(txts['author']).most_common(20)).keys():
            #    continue
            txt, title = ' '.join(TextBlob(strip_headers(load_etext(int(i)))).lower().words), list(get_metadata('title', i))[0]
            #print(txt,title)
            txts['text'].append(txt)
            txts['author'].append(author)
            txts['title'].append(title)
        except:
            pass
    #if txts["author"] and all([count >= 15 for author,count in Counter(txts['author']).most_common(7)]): break#len(txts['title']) > 300: break


In [4]:
all([count >= 7 for author,count in Counter(txts['author']).most_common(7)])
#[len(txts[x]) for x in ['author','title','text']]
#all([])

True

In [34]:
txts['author']

['Stevenson, Robert Louis',
 'Stevenson, Robert Louis',
 'Dickens, Charles',
 'Dickens, Charles',
 'Shakespeare, William',
 'Doyle, Arthur Conan',
 'Stevenson, Robert Louis',
 'Doyle, Arthur Conan',
 'Stevenson, Robert Louis',
 'Doyle, Arthur Conan',
 'James, Henry',
 'James, Henry',
 'James, Henry',
 'James, Henry',
 'James, Henry',
 'James, Henry',
 'James, Henry',
 'James, Henry',
 'London, Jack',
 'Doyle, Arthur Conan',
 'Doyle, Arthur Conan',
 'Stevenson, Robert Louis',
 'Stevenson, Robert Louis',
 'Doyle, Arthur Conan',
 'Doyle, Arthur Conan',
 'Jerome, Jerome K. (Jerome Klapka)',
 'London, Jack',
 'London, Jack',
 'Stevenson, Robert Louis',
 'Davis, Richard Harding',
 'Stevenson, Robert Louis',
 'Davis, Richard Harding',
 'Stevenson, Robert Louis',
 'Stevenson, Robert Louis',
 'Doyle, Arthur Conan',
 'Doyle, Arthur Conan',
 'Davis, Richard Harding',
 'Stevenson, Robert Louis',
 'Stevenson, Robert Louis',
 'Stevenson, Robert Louis',
 'Stevenson, Robert Louis',
 'Stevenson, Robert

In [35]:
txts['text'] = np.array(txts['text'])
txts['author'] = np.array(txts['author'])
txts['title'] = np.array(txts['title'])

In [38]:
# grab most prevalent authors
top_authors = np.array([x[0] for x in Counter(txts['author']).most_common()])#[:10]
top_authors
print(len(txts['author']),len(txts['title']),len(txts['text']))
Counter(txts['author'][np.isin(txts['author'],top_authors)])

700 700 700


Counter({'Stevenson, Robert Louis': 39,
         'Dickens, Charles': 50,
         'Shakespeare, William': 156,
         'Doyle, Arthur Conan': 28,
         'James, Henry': 37,
         'London, Jack': 33,
         'Jerome, Jerome K. (Jerome Klapka)': 28,
         'Davis, Richard Harding': 28,
         'Balzac, Honoré de': 93,
         'Dumas, Alexandre': 32,
         'Goethe, Johann Wolfgang von': 32,
         'Harte, Bret': 49,
         'Huxley, Thomas Henry': 31,
         'Galsworthy, John': 33,
         'Casanova, Giacomo': 31})

In [40]:
text2author = dict(zip(txts['title'],txts['author']))
txts_df=pd.DataFrame.from_dict(txts)
txts_main = txts_df[np.isin(txts['author'],top_authors)]
g = txts_main.groupby('author')
txts_main_balanced = g.apply(lambda x: x.sample(g.size().min())).reset_index(drop=True)

In [41]:
txts_main_balanced.to_pickle("./texts.pkl")

In [42]:
txts_main_balanced = pd.read_pickle("./texts.pkl")

,title,text
author,,
"Balzac, Honoré de",28,28
"Casanova, Giacomo",28,28
"Davis, Richard Harding",28,28
"Dickens, Charles",28,28
"Doyle, Arthur Conan",28,28
"Dumas, Alexandre",28,28
"Galsworthy, John",28,28
"Goethe, Johann Wolfgang von",28,28
"Harte, Bret",28,28


In [108]:
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer.fit(txts_main_balanced['text'])#, TfidfTransformer remove numbers
initial_count_matrix = count_vectorizer.transform(txts_main_balanced['text'])
"""tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(txts_main['text'])"""
feature_names = np.array(count_vectorizer.get_feature_names())
count_matrix = TfidfTransformer().fit_transform(initial_count_matrix)

In [109]:
transformer_pipeline = Pipeline([('pca',KPCA(n_components=3,kernel='linear'))])#,('umap',UMAP(n_neighbors=10, n_components=3, random_state=42))][:-1])
t_data = transformer_pipeline.fit_transform(count_matrix)
t_data_df = pd.DataFrame(np.vstack((txts_main_balanced['author'].values,txts_main_balanced['title'].values,t_data.T)).T,columns=['author','title','x','y','z'])
plotly_plot(t_data_df, 'output.html', False, 'title', 'author')

In [110]:
from sklearn import preprocessing

n_authors = len(t_data_df['author'].unique())
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(t_data_df['author'].unique())
y_true = label_encoder.transform(t_data_df['author'])

In [111]:
from sklearn.cluster import SpectralClustering

y_cluster = SpectralClustering(n_clusters=6,n_neighbors=5, affinity='nearest_neighbors').fit_predict(t_data_df[['x','y','z']])
t_data_df['cluster'] = y_cluster
plotly_plot(t_data_df, 'cluster.html', False, 'title', 'cluster')

/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning:

Graph is not fully connected, spectral embedding may not work as expected.



In [112]:
from sklearn.feature_selection import SelectKBest, chi2
ch2 = SelectKBest(chi2, k='all')
ch2.fit(count_matrix, y_cluster)
top_scores_bool = np.isin(ch2.scores_,np.sort(ch2.scores_)[-500:])
selected_features = feature_names[top_scores_bool]
count_matrix_selected = TfidfTransformer().fit_transform(initial_count_matrix[:,top_scores_bool]) # address class balancing

In [125]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(n_estimators=100)
X_train, X_test, y_train, y_test = train_test_split(count_matrix_selected, y_true, test_size=0.33, random_state=42)
#rf.fit(X_train, y_train)
rf.fit(count_matrix_selected, y_true)
#y_pred = rf.predict(X_test)
#sum(y_pred==y_test)/float(len(y_test))
rf_feature_importance_bool = np.isin(rf.feature_importances_,np.sort(rf.feature_importances_)[-50:])
list(zip(selected_features[rf_feature_importance_bool],rf.feature_importances_[rf_feature_importance_bool]))
# Maybe compare to tfidf scores

[('10', 0.0066395735455272945),
 ('11', 0.006742889856593597),
 ('13', 0.0042040276576479066),
 ('14', 0.004940918850954292),
 ('16', 0.005584694236849917),
 ('17', 0.005787966110478521),
 ('20', 0.004754051303974192),
 ('21', 0.002660066902553633),
 ('23', 0.00209101208684322),
 ('24', 0.0020879193247051813),
 ('26', 0.009608238526598218),
 ('27', 0.0018455373283497879),
 ('29', 0.0012721991764633519),
 ('30', 0.003979733063525963),
 ('31', 0.0014941848077582504),
 ('32', 0.0017892098191957917),
 ('33', 0.0015045143317587472),
 ('38', 0.0014472126480974785),
 ('40', 0.002486299195158812),
 ('41', 0.0027521262441929438),
 ('43', 0.0008113587017748696),
 ('44', 0.00076362504239658),
 ('46', 0.0012935016510309864),
 ('agesilaus', 0.00939910565997488),
 ('altra', 0.0008995825699361944),
 ('ap', 0.001611811234033456),
 ('aristot', 0.0012140596498339583),
 ('art', 0.0097380857510324),
 ('athenians', 0.0032835206020587924),
 ('bastard', 0.0021085087410216263),
 ('beam', 0.0019366133659619268

In [128]:
final_count_matrix = (TfidfTransformer().fit_transform(initial_count_matrix[:,top_scores_bool][:,rf_feature_importance_bool]))#preprocessing.StandardScaler(with_mean=False).fit_transform
transformer_pipeline = Pipeline([('pca',KPCA(n_components=3,kernel='linear'))])#,('umap',UMAP(n_neighbors=10, n_components=3, random_state=42))][:-1])
t_data = transformer_pipeline.fit_transform(final_count_matrix)
t_data_df = pd.DataFrame(np.vstack((txts_main_balanced['author'].values,txts_main_balanced['title'].values,t_data.T)).T,columns=['author','title','x','y','z'])
plotly_plot(t_data_df, 'output_final.html', False, 'title', 'author')

In [ ]:
# visualize, cluster, then run classifier after initial feature selection and use RF or chi2 to assign importance for last round selection
# then add plot of words versus authors and dimensionality reduce words
# final app has words versus authors box plots, PCA words and PCA books
# cluster
# chi2 on large matrix
# RF
# final feature select
# 


In [23]:
#blob.lower().words
# Make a sparsedataframe that is indexed by book name
#blob = TextBlob(text)
#print(list(metadata)[0])
#final_count_matrix = pd.SparseDataFrame(count_matrix,index=txts['title'])
# may just want to go by author, and choose a few, run get_etexts
#print(get_metadata('title', 2701))
#dir(get_metadata)
text = strip_headers(load_etext(2701)).strip()

In [4]:
def plotly_plot(t_data_df, output_fname, axes_off=False, text_column='text', color_column=None):
    plots = []
    if color_column == None:
        plots.append(
            go.Scatter3d(x=t_data_df['x'], y=t_data_df['y'],
                         z=t_data_df['z'],
                         name='', mode='markers',
                         marker=dict(color=t_data_df[color_column], size=2, colorscale='Viridis',
                         colorbar=dict(title='Colorbar')), text=t_data_df[text_column]))
    else:
        
        colors = t_data_df[color_column].unique()
        c = ['hsl(' + str(h) + ',50%' + ',50%)' for h in np.linspace(0, 360, len(colors) + 2)]
        color_dict = {name: c[i] for i,name in enumerate(sorted(colors))}

        for name,col in color_dict.items():
            plots.append(
                go.Scatter3d(x=t_data_df['x'][t_data_df[color_column]==name], y=t_data_df['y'][t_data_df[color_column]==name],
                             z=t_data_df['z'][t_data_df[color_column]==name],
                             name=str(name), mode='markers',
                             marker=dict(color=col, size=2), text=t_data_df[text_column][t_data_df[color_column]==name]))
    if axes_off:
        fig = go.Figure(data=plots,layout=go.Layout(scene=dict(xaxis=dict(title='',autorange=True,showgrid=False,zeroline=False,showline=False,ticks='',showticklabels=False),
            yaxis=dict(title='',autorange=True,showgrid=False,zeroline=False,showline=False,ticks='',showticklabels=False),
            zaxis=dict(title='',autorange=True,showgrid=False,zeroline=False,showline=False,ticks='',showticklabels=False))))
    else:
        fig = go.Figure(data=plots)
    py.plot(fig, filename=output_fname, auto_open=False)
    

In [23]:
dir(umap)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [16]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )